In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and prepare the datasets
data1 = pd.read_csv("../dataset/processed_data/Parkinson_CV_ensemble.tab", delimiter="\t")
data2 = pd.read_csv("../dataset/processed_data/Parkinson_FD_ensemble_pca.tab", delimiter="\t")

# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]
label1 = label1.replace({'Co': 0, 'Pt': 1})
label2 = label2.replace({'Co': 0, 'Pt': 1})

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]

# Combine the datasets and labels
data = pd.concat([data1, data2], axis=1)
label = label1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (X_train.shape[1],)

# Define the attention mechanism layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Define the model architecture with attention mechanism
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    hidden = tf.keras.layers.Dense(32, activation='relu')(x)
    context_vector, attention_weights = Attention(32)(x, hidden)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(context_vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the model
model = create_model(input_shape)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=4, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', accuracy)





Epoch 1/30
54/54 [==============================] - 0s 2ms/step - loss: 134.3965 - accuracy: 0.6481 - val_loss: 230.2763 - val_accuracy: 0.5185
Epoch 2/30
54/54 [==============================] - 0s 889us/step - loss: 116.5293 - accuracy: 0.5787 - val_loss: 177.9948 - val_accuracy: 0.4815
Epoch 3/30
54/54 [==============================] - 0s 798us/step - loss: 107.7457 - accuracy: 0.5370 - val_loss: 146.2801 - val_accuracy: 0.5370
Epoch 4/30
54/54 [==============================] - 0s 2ms/step - loss: 92.2424 - accuracy: 0.5324 - val_loss: 126.9870 - val_accuracy: 0.5000
Epoch 5/30
54/54 [==============================] - 0s 819us/step - loss: 97.2903 - accuracy: 0.5787 - val_loss: 133.6521 - val_accuracy: 0.4630
Epoch 6/30
54/54 [==============================] - 0s 810us/step - loss: 83.9109 - accuracy: 0.5509 - val_loss: 118.9699 - val_accuracy: 0.5370
Epoch 7/30
54/54 [==============================] - 0s 789us/step - loss: 82.5150 - accuracy: 0.5880 - val_loss: 117.4286 - val_acc

In [10]:
print(X_test.shape)
print(y_test.shape)

(54, 32)
(54,)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 4.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [7]:
import pandas as pd

# Read the CSV file into a DataFrame
data1 = pd.read_csv('../dataset/processed_data/Parkinson_CV_merged.tab',sep='\t')
data2 = pd.read_csv('../dataset/processed_data/Parkinson_FD_merged.tab',sep='\t')
# Combine the datasets and labels
# Filter the last column as the label
label1 = data1.iloc[:, -1]
label2 = data2.iloc[:, -1]

# Remove the last column from the datasets
data1 = data1.iloc[:, :-1]
data2 = data2.iloc[:, :-1]
# label1 = label1.replace({'Co': 0, 'Pt': 1})
# label2 = label2.replace({'Co': 0, 'Pt': 1})
X_test = pd.concat([data1, data2], axis=1)
y_test = label1
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

InvalidArgumentError: Graph execution error:

Detected at node 'model_1/dense_6/Relu' defined at (most recent call last):
    File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/lib/python3.10/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/opt/homebrew/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/homebrew/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio